---
jupyter: python3
---

In [ ]:
!pip install icecream
!pip uninstall -y kagglehub && pip install kagglehub


from typing import Final
from icecream import ic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import kagglehub


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

kaggle_path = (
    kagglehub.dataset_download(
        "swathiunnikrishnan/amazon-consumer-behaviour-dataset"
    )
)

PATH: Final = os.path.join(kaggle_path, 'Amazon Customer Behavior Survey.csv')

df = pd.read_csv(PATH)
df['id'] = df.index + 1

In [ ]:
def save_plot(file_name: str):
    """
        Para guardar en fotos los gráficos
    """
    plt.savefig(file_name, bbox_inches='tight')

In [ ]:
df.head()

In [ ]:
len(df.columns)

In [ ]:
total_registros = len(df)

In [ ]:
ver_luego = df.iloc[:, -9:-1]

In [ ]:
for col in ver_luego.columns:
    ic(col)
    ic(ver_luego[col].unique())

In [ ]:
for col in ver_luego.columns:
    ic(col)
    ic(ver_luego[col].nunique())

In [ ]:
for col in ver_luego.columns:
    ic(col)
    ic(ver_luego[col].value_counts())

In [ ]:
"""
TODO:
[X] - Idea de Limpiar Purchase_Categories
[X] - Probar Diagrama de Venn entre los más votados como la kc :P
    [X] - Usar ese método de armar columnas bool como la del profe alejo :P
[X] - ver si el valor de total cambiar por total_registros en el diagrama de venn
"""

In [ ]:
df_copy = df.copy()
df_copy['Purchase_Categories'] = df_copy['Purchase_Categories'].str.split(';')
df_copy = df_copy.explode('Purchase_Categories').reset_index(drop=True)
len(df_copy)

In [ ]:
df_copy['Purchase_Categories'].value_counts()

In [ ]:
from matplotlib.patches import FancyBboxPatch


category_counts = df_copy['Purchase_Categories'].value_counts()

colors_top_five = ['#008000', '#808080', '#808080', '#808080', '#808080']

title_text: str = 'Preferencia de compras'


plt.figure(figsize=(6, 4))


ax = sns.barplot(
    x=category_counts.values,
    y=category_counts.index, orient='h',
    joinstyle='bevel'
)


new_patches = []
for patch, color, purchase_category, total, in zip(
    ax.patches, colors_top_five,
    category_counts.values, category_counts.values
):

    bb = patch.get_bbox()
    p_bbox = FancyBboxPatch(
        (bb.xmin, bb.ymin), abs(bb.width), abs(bb.height),
        boxstyle='round,pad=-0.05,rounding_size=0.73',
        ec='none', fc=color, mutation_aspect=0.73
    )
    patch.remove()
    new_patches.append(p_bbox)

    ax.annotate(
        f'{(purchase_category*100/total_registros):.0f}%', xy=(40, patch.get_y() + patch.get_height()/2),
        xytext=(-5,0), textcoords='offset points',
        arrowprops=dict(arrowstyle='-', color='none'),
        color='white', fontweight='bold', fontsize=12, ha='right', va='center',
        xycoords='data',
        bbox=dict(facecolor='none', edgecolor='none', pad=0),
        annotation_clip=False
    )

    ax.annotate(
        f'{total}',
        xy=(patch.get_width(), patch.get_y() + patch.get_height()/2),
        xytext=(-5,0), textcoords='offset points',
        arrowprops=dict(arrowstyle='-', color='none'),
        color='white', fontweight='bold', fontsize=12, ha='right', va='center',
        xycoords='data',
        bbox=dict(facecolor='none', edgecolor='none', pad=0),
        annotation_clip=False
    )

for patch in new_patches:
    ax.add_patch(patch)

ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticks([])
ax.yaxis.grid(False)
ax.xaxis.grid(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.title(
    title_text,
    fontsize=18, fontweight='bold', x=0.35
)

ax.text(
    0.98, 0.02, f'{total_registros}\nRegistros totales',
    ha='right', va='bottom', transform=ax.transAxes,
    fontsize=12, fontweight='bold'
)

save_plot(f'{title_text}.png')
plt.show()

In [ ]:
df['Purchase_Categories'] = df['Purchase_Categories'].str.split(';')
df = df.explode('Purchase_Categories').reset_index(drop=True)

df_dummies = pd.get_dummies(df['Purchase_Categories'])
#df_combined = df.join(df_dummies).groupby('Timestamp').max().reset_index()
df_combined = df.join(df_dummies).groupby('id').max().reset_index()
df_combined

In [ ]:
df_filtered = df_combined[['id'] + df_combined.columns[-5:].tolist()]
df_filtered.columns

In [ ]:
!pip install matplotlib-venn

In [ ]:
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import fontManager, FontProperties
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib_venn import venn3

In [ ]:
def purchases(category: str, df: pd.DataFrame=df_filtered):
    '''
        Para filtrar sus usuarios únicos de cada purchases
    '''
    return df.query(f'`{category}` == True')['id'].drop_duplicates()

In [ ]:
purchases_list_top_three = category_counts.index[:3].tolist()


categories_top_one = purchases(purchases_list_top_three[0])
categories_top_two = purchases(purchases_list_top_three[1])
categories_top_three = purchases(purchases_list_top_three[2])

set_top_one = set(categories_top_one)
set_top_two = set(categories_top_two)
set_top_three = set(categories_top_three)

# Calcular los tamaños de los conjuntos
size_100 = len(set_top_one - (set_top_two | set_top_three))
size_010 = len(set_top_two - (set_top_one | set_top_three))
size_001 = len(set_top_three - (set_top_one | set_top_two))
size_110 = len((set_top_one & set_top_two) - set_top_three)
size_101 = len((set_top_one & set_top_three) - set_top_two)
size_011 = len((set_top_two & set_top_three) - set_top_one)
size_111 = len(set_top_one & set_top_two & set_top_three)

# Calcular los porcentajes
total = sum(
    [size_100, size_010, size_001, size_110, size_101, size_011, size_111]
)

percent_100 = (size_100 / total_registros) * 100
percent_010 = (size_010 / total_registros) * 100
percent_001 = (size_001 / total_registros) * 100
percent_110 = (size_110 / total_registros) * 100
percent_101 = (size_101 / total_registros) * 100
percent_011 = (size_011 / total_registros) * 100
percent_111 = (size_111 / total_registros) * 100

In [ ]:
people_sum: int = int(df_filtered.id.drop_duplicates().count())

count_com = [total, int(people_sum - total)]
labels_com = "Están en\nlas más \nsolicitadas", "No están\nen las más\nsolicitadas"
pastel_colors = ['#D291BC', '#BDFCFE']

title_text: str = "Relación de categorías de compra"

fig, ax = plt.subplots(figsize=(10, 8))

wedges, texts, autotexts = ax.pie(
    count_com, labels=labels_com, autopct="%1.1f%%",
    colors=pastel_colors,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1, 'linestyle': 'solid'},
    pctdistance=0.75,
    textprops={'fontsize': 22}
)

centre_circle = plt.Circle((0, 0), 0.4, fc='white')
ax.add_artist(centre_circle)

ax.text(0, 0, f'{people_sum}\ncompradores', ha='center', va='center', fontsize=21)

ax.set_title(
    title_text,
    fontsize=28,
    fontweight="bold"
)

save_plot(f'{title_text}.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

venn = venn3(
    [set_top_one, set_top_two, set_top_three],
    set_labels=(
        purchases_list_top_three[0],
        purchases_list_top_three[1],
        purchases_list_top_three[2]
    )
)

venn.get_label_by_id('100').set_text(
    f"{size_100}\n{percent_100:.2f}%")
venn.get_label_by_id('010').set_text(
    f"{size_010}\n{percent_010:.2f}%")
venn.get_label_by_id('001').set_text(
    f"{size_001}\n{percent_001:.2f}%")
venn.get_label_by_id('110').set_text(
    f"{size_110}\n{percent_110:.2f}%")
venn.get_label_by_id('101').set_text(
    f"{size_101}\n{percent_101:.2f}%")
venn.get_label_by_id('011').set_text(
    f"{size_011}\n{percent_011:.2f}%")
venn.get_label_by_id('111').set_text(
    f"{size_111}\n{percent_111:.2f}%")


colors = sns.color_palette('Set3', 7)
for patch, color in zip(venn.patches, colors):
    patch.set_facecolor(color)


for label in venn.set_labels:
    label.set_fontsize(20)

for label in venn.subset_labels:
    if label:
        label.set_fontsize(16)

ax.set_title(
    'Gráfico de Venn',
    fontsize=28,
    fontweight="bold"
)

save_plot('Gráfico de Venn.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))


unique_category = size_100 + size_010 + size_001
two_categories = size_110 + size_101 + size_011

count_duelists = [unique_category, two_categories, size_111]
labels = "Una\nsola", "En dos", "Están\nen\nlas 3"
pastel_colors = ['#92c6ff', '#ffb7ce', '#b7e3cc']


wedges, texts, autotexts = ax.pie(
    count_duelists, labels=labels, autopct="%1.1f%%",
    colors=pastel_colors,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1, 'linestyle': 'solid'},
    pctdistance=0.75,
    textprops={'fontsize': 20}
)

centre_circle = plt.Circle((0, 0), 0.4, fc='white')
ax.add_artist(centre_circle)

ax.text(0, 0, f'{total}\ncompradores', ha='center', va='center', fontsize=21)

ax.set_title(
    'De las 3, en cuántas\ncategorías de compras están',
    fontsize=24,
    fontweight="bold"
)

save_plot('Gustos en compras.png')
plt.show()

In [ ]:
service_appreciatione = df['Service_Appreciation'].value_counts().reset_index()

services_top_forth = service_appreciatione.iloc[:4]

otros_service = service_appreciatione.iloc[4:].sum()
otros_service['Service_Appreciation'] = 'Others'
count_services_app = pd.concat(
    [services_top_forth, otros_service.to_frame().T],
    ignore_index=True
)
count_services_app

In [ ]:
plt.figure(figsize=(6, 4))

colors_top_five = ['#4169E1', '#808080', '#808080', '#808080', '#808080']

ax = sns.barplot(
    x=count_services_app['count'],
    y=count_services_app['Service_Appreciation'], orient='h',
    joinstyle='bevel'
)


new_patches = []
for patch, color, total, in zip(
    ax.patches, colors_top_five, count_services_app['count']
):

    bb = patch.get_bbox()
    p_bbox = FancyBboxPatch(
        (bb.xmin, bb.ymin), abs(bb.width), abs(bb.height),
        boxstyle='round,pad=-0.05,rounding_size=0.73',
        ec='none', fc=color, mutation_aspect=0.73
    )
    patch.remove()
    new_patches.append(p_bbox)

    ax.annotate(
        f'{total}',
        xy=(patch.get_width(), patch.get_y() + patch.get_height()/2),
        xytext=(-5,0), textcoords='offset points',
        arrowprops=dict(arrowstyle='-', color='none'),
        color='white', fontweight='bold', fontsize=12, ha='right', va='center',
        xycoords='data',
        bbox=dict(facecolor='none', edgecolor='none', pad=0),
        annotation_clip=False
    )

for patch in new_patches:
    ax.add_patch(patch)

ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticks([])
ax.yaxis.grid(False)
ax.xaxis.grid(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.title(
    'Service Appreciation',
    fontsize=14, fontweight='bold', x=0.35
)

ax.text(
    0.98, 0.02, f'{total_registros}\nRegistros totales',
    ha='right', va='bottom', transform=ax.transAxes,
    fontsize=12, fontweight='bold'
)

save_plot('Service Appreciation.png')
plt.show()

In [ ]:
improven_areas_series = df['Improvement_Areas'].value_counts().reset_index()

top_forth_improvenment = improven_areas_series.iloc[:4]

otros_improven = improven_areas_series.iloc[4:].sum()
otros_improven['Improvement_Areas'] = 'Others'
improvement_areas = pd.concat(
    [top_forth_improvenment, otros_improven.to_frame().T],
    ignore_index=True
)
improvement_areas

In [ ]:
plt.figure(figsize=(6, 4))

colors_top_five = ['#FF00FF', '#808080', '#808080', '#808080', '#808080']

ax = sns.barplot(
    x=improvement_areas['count'],
    y=improvement_areas['Improvement_Areas'], orient='h',
    joinstyle='bevel'
)


new_patches = []
for patch, color, total, in zip(
    ax.patches, colors_top_five, improvement_areas['count']
):

    bb = patch.get_bbox()
    p_bbox = FancyBboxPatch(
        (bb.xmin, bb.ymin), abs(bb.width), abs(bb.height),
        boxstyle='round,pad=-0.05,rounding_size=0.73',
        ec='none', fc=color, mutation_aspect=0.73
    )
    patch.remove()
    new_patches.append(p_bbox)

    ax.annotate(
        f'{total}',
        xy=(patch.get_width(), patch.get_y() + patch.get_height()/2),
        xytext=(-5,0), textcoords='offset points',
        arrowprops=dict(arrowstyle='-', color='none'),
        color='white', fontweight='bold', fontsize=12, ha='right', va='center',
        xycoords='data',
        bbox=dict(facecolor='none', edgecolor='none', pad=0),
        annotation_clip=False
    )

for patch in new_patches:
    ax.add_patch(patch)

ax.set_xlabel('')
ax.set_ylabel('')
ax.set_xticks([])
ax.yaxis.grid(False)
ax.xaxis.grid(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.title(
    'Improvement Areas',
    fontsize=14, fontweight='bold', x=0.35
)

ax.text(
    0.98, 0.02, f'{total_registros}\nRegistros totales',
    ha='right', va='bottom', transform=ax.transAxes,
    fontsize=12, fontweight='bold'
)


save_plot('Improvement Areas.png')
plt.show()

In [ ]:
ver_mas_tarde = ver_luego.iloc[:, :-2]

colors_top  = ['#FF6347', '#0000FF', '#8A2BE2', '#40E0D0', '#FF69B4', '#DC143C']

for index, col in enumerate(ver_mas_tarde.columns):
    ic(col)
    plt.figure(figsize=(10, 6))

    ax = sns.countplot(
        y=ver_mas_tarde[col],
        order=ver_mas_tarde[col].value_counts().index
    )

    new_patches = []

    colors_top_five = [colors_top[index], '#808080', '#808080', '#808080', '#808080']

    for patch, color, total, in zip(
        ax.patches, colors_top_five, ver_mas_tarde[col].value_counts()
    ):

        bb = patch.get_bbox()
        p_bbox = FancyBboxPatch(
            (bb.xmin, bb.ymin), abs(bb.width), abs(bb.height),
            boxstyle='round,pad=-0.05,rounding_size=0.73',
            ec='none', fc=color, mutation_aspect=0.73
        )
        patch.remove()
        new_patches.append(p_bbox)

        ax.annotate(
            f'{total}',
            xy=(patch.get_width(), patch.get_y() + patch.get_height()/2),
            xytext=(-5,0), textcoords='offset points',
            arrowprops=dict(arrowstyle='-', color='none'),
            color='white', fontweight='bold', fontsize=12, ha='right', va='center',
            xycoords='data',
            bbox=dict(facecolor='none', edgecolor='none', pad=0),
            annotation_clip=False
        )

    for patch in new_patches:
        ax.add_patch(patch)


    ax.set_title(f'{col}', fontsize=16, fontweight='bold')
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.set_xticklabels([])
    ax.yaxis.grid(False)
    ax.xaxis.grid(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    save_plot(f'{col}.png')
    plt.show()